In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [16]:
from datetime import datetime

In [2]:
deposit_df=pd.read_excel("C:\\Users\\Asus\\Contacts\\Vindiata Consulting\\depositdata.xlsx")

In [3]:
deposit_df.head(5)

,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500


In [11]:
deposit_df

,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500
...,...,...,...
17433,654,31-10-2022 23:57,1200
17434,980,31-10-2022 23:58,200
17435,2,31-10-2022 23:58,40000
17436,612,31-10-2022 23:58,2800


In [4]:
withdrawal_df = pd.read_excel("C:\\Users\\Asus\\Contacts\\Vindiata Consulting\\withdrawaldata.xlsx")

In [5]:
withdrawal_df.head(5)

,User Id,Datetime,Amount
0,190,2022-01-10 00:03:00,5872
1,159,2022-01-10 00:16:00,9540
2,164,2022-01-10 00:24:00,815
3,946,2022-01-10 00:29:00,23000
4,763,2022-01-10 00:40:00,9473


In [6]:
games_df=pd.read_excel("C:\\Users\\Asus\\Contacts\\Vindiata Consulting\\gameplaydata.xlsx")

In [7]:
games_df.head(5)

,User ID,Games Played,Datetime
0,851,1,2022-01-10 00:00:00
1,717,1,2022-01-10 00:00:00
2,456,1,2022-01-10 00:00:00
3,424,1,2022-01-10 00:00:00
4,845,1,2022-01-10 00:00:00


In [18]:
games_df['Datetime'] = pd.to_datetime(games_df['Datetime'])


In [19]:
withdrawal_df['Datetime'] = pd.to_datetime(withdrawal_df['Datetime'])
deposit_df['Datetime'] = pd.to_datetime(deposit_df['Datetime'])

In [20]:
withdrawal_df

,User Id,Datetime,Amount
0,190,2022-01-10 00:03:00,5872
1,159,2022-01-10 00:16:00,9540
2,164,2022-01-10 00:24:00,815
3,946,2022-01-10 00:29:00,23000
4,763,2022-01-10 00:40:00,9473
...,...,...,...
3561,559,2022-10-31 23:27:00,5000
3562,407,2022-10-31 23:51:00,3000
3563,389,2022-10-31 23:56:00,14481
3564,11,2022-10-31 23:57:00,4000


In [21]:
deposit_df

,User Id,Datetime,Amount
0,357,2022-01-10 00:03:00,2000
1,776,2022-01-10 00:03:00,2500
2,492,2022-01-10 00:06:00,5000
3,803,2022-01-10 00:07:00,5000
4,875,2022-01-10 00:09:00,1500
...,...,...,...
17433,654,2022-10-31 23:57:00,1200
17434,980,2022-10-31 23:58:00,200
17435,2,2022-10-31 23:58:00,40000
17436,612,2022-10-31 23:58:00,2800


In [22]:
deposit_df['Datetime'] = pd.to_datetime(deposit_df['Datetime'], format='%Y-%m-%d %H:%M:%S')
withdrawal_df['Datetime'] = pd.to_datetime(withdrawal_df['Datetime'], format='%Y-%m-%d %H:%M:%S')
games_df['Datetime'] = pd.to_datetime(games_df['Datetime'], format='%Y-%m-%d %H:%M:%S')

In [23]:
# Function to filter data by slot
def get_slot_data(df, date, slot):
    """Filters data for a given date and slot."""
    if slot == 'S1':
        start, end = datetime(date.year, date.month, date.day, 0, 0), datetime(date.year, date.month, date.day, 11, 59)
    else:  # Slot S2
        start, end = datetime(date.year, date.month, date.day, 12, 0), datetime(date.year, date.month, date.day, 23, 59)
    
    return df[(df['Datetime'] >= start) & (df['Datetime'] <= end)]


In [24]:
# Function to calculate loyalty points
def calculate_loyalty_points(user_id, games_played, deposit_amount, withdrawal_amount, deposit_count, withdrawal_count):
    points = (0.01 * deposit_amount) + \
             (0.005 * withdrawal_amount) + \
             (0.001 * max(deposit_count - withdrawal_count, 0)) + \
             (0.2 * games_played)
    return points

## 1. Player-wise loyalty points for specified slots

In [33]:
# 1. Player-wise loyalty points for specified slots
slot_dates = [('02-10-2022', 'S1'), ('16-10-2022', 'S2'), ('18-10-2022', 'S1'), ('26-10-2022', 'S2')]
results = []

In [34]:
for date_str, slot in slot_dates:
    date = datetime.strptime(date_str, '%d-%m-%Y')
    # Filter data for slot
    games_slot = get_slot_data(games_df, date, slot)
   
    deposits_slot = get_slot_data(deposit_df, date, slot)
    withdrawals_slot = get_slot_data(withdrawal_df, date, slot)
    # Calculate points for each player in slot
    for user_id in set(games_slot['User ID']).union(deposits_slot['User Id'], withdrawals_slot['User Id']):
        games_played = games_slot[games_slot['User ID'] == user_id]['Games Played'].sum()
        deposit_amount = deposits_slot[deposits_slot['User Id'] == user_id]['Amount'].sum()
        withdrawal_amount = withdrawals_slot[withdrawals_slot['User Id'] == user_id]['Amount'].sum()
        deposit_count = deposits_slot[deposits_slot['User Id'] == user_id].shape[0]
        withdrawal_count = withdrawals_slot[withdrawals_slot['User Id'] == user_id].shape[0]
        # Calculate loyalty points
        points = calculate_loyalty_points(user_id, games_played, deposit_amount, withdrawal_amount, deposit_count, withdrawal_count)
        
        results.append({'User Id': user_id, 'Date': date_str, 'Slot': slot, 'Loyalty Points': points})

In [35]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
print("Loyalty Points for specified slots:\n", results_df)

Loyalty Points for specified slots:
       User Id        Date Slot  Loyalty Points
0           2  16-10-2022   S2           0.400
1           5  16-10-2022   S2          11.802
2           6  16-10-2022   S2           0.200
3           8  16-10-2022   S2           1.000
4           9  16-10-2022   S2          30.201
...       ...         ...  ...             ...
1839      992  26-10-2022   S2         158.800
1840      995  26-10-2022   S2           0.200
1841      996  26-10-2022   S2           0.800
1842      997  26-10-2022   S2           0.400
1843      999  26-10-2022   S2           0.200

[1844 rows x 4 columns]


## Calculate overall loyalty points earned and rank players on the basis of loyalty points in the month of October. 
   

In [36]:
# Aggregate data for each user for the entire month of October
october_start = datetime(2022, 10, 1)
october_end = datetime(2022, 10, 31, 23, 59)

In [37]:
# Filter the data for October only
games_october = games_df[(games_df['Datetime'] >= october_start) & (games_df['Datetime'] <= october_end)]
deposits_october = deposit_df[(deposit_df['Datetime'] >= october_start) & (deposit_df['Datetime'] <= october_end)]
withdrawals_october = withdrawal_df[(withdrawal_df['Datetime'] >= october_start) & (withdrawal_df['Datetime'] <= october_end)]

In [38]:
# Initialize a dictionary to store loyalty points and games played for each user
user_points = {}


In [43]:
# Calculate loyalty points for each unique user in the data
for user_id in set(games_october['User ID']).union(deposits_october['User Id'], withdrawals_october['User Id']):
    # Get total games played, deposit, and withdrawal amounts for each user
    games_played = games_october[games_october['User ID'] == user_id]['Games Played'].sum()
    deposit_amount = deposits_october[deposits_october['User Id'] == user_id]['Amount'].sum()
    withdrawal_amount = withdrawals_october[withdrawals_october['User Id'] == user_id]['Amount'].sum()
    deposit_count = deposits_october[deposits_october['User Id'] == user_id].shape[0]
    withdrawal_count = withdrawals_october[withdrawals_october['User Id'] == user_id].shape[0]
    
    # Calculate loyalty points
    points = calculate_loyalty_points(user_id, games_played, deposit_amount, withdrawal_amount, deposit_count, withdrawal_count)
    
    # Store the results in a dictionary
    user_points[user_id] = {
        'Loyalty Points': points,
        'Games Played': games_played
    }

In [44]:
user_points_df = pd.DataFrame.from_dict(user_points, orient='index').reset_index()
user_points_df.rename(columns={'index': 'User_Id'}, inplace=True)

# Rank users by loyalty points and, in case of a tie, by the number of games played
user_points_df = user_points_df.sort_values(by=['Loyalty Points', 'Games Played'], ascending=[False, False]).reset_index(drop=True)

# Display ranked users
print("Ranked Users by Loyalty Points for October:\n", user_points_df)

Ranked Users by Loyalty Points for October:
      User_Id  Loyalty Points  Games Played
0        634       61121.160            22
1        714       14790.824             4
2        212       13947.217             0
3        672       13238.624             8
4         99       12469.948             4
..       ...             ...           ...
991       49           0.200             1
992      388           0.200             1
993      507           0.200             1
994      546           0.200             1
995      771           0.200             1

[996 rows x 3 columns]


## 3. What is the average deposit amount?

In [45]:
average_deposit_amount = deposit_df['Amount'].mean()

print("Average Deposit Amount:", average_deposit_amount)

Average Deposit Amount: 5492.185399701801


## 4. What is the average deposit amount per user in a month?

In [46]:
# Group by 'User Id' and calculate total deposit per user
user_total_deposits = deposit_df.groupby('User Id')['Amount'].sum()

# Calculate the average deposit amount per user
average_deposit_per_user = user_total_deposits.mean()

print("Average Deposit Amount Per User in the Month:", average_deposit_per_user)


Average Deposit Amount Per User in the Month: 104669.64918032786


## 5. What is the average number of games played per user?

In [48]:
# Group by 'User Id' and calculate total games played per user
user_total_games = games_df.groupby('User ID')['Games Played'].sum()

# Calculate the average number of games played per user
average_games_per_user = user_total_games.mean()

print("Average Number of Games Played Per User:", average_games_per_user)


Average Number of Games Played Per User: 355.267


##  How much bonus should be allocated to leaderboard players?

In [54]:
# Calculate loyalty points
loyalty_points = {}

# Calculate the loyalty points for each player
for user_id in set(games_df['User ID']).union(deposit_df['User Id'], withdrawal_df['User Id']):
    games_played = games_df[games_df['User ID'] == user_id]['Games Played'].sum()
    deposit_amount = deposit_df[deposit_df['User Id'] == user_id]['Amount'].sum()
    withdrawal_amount = withdrawal_df[withdrawal_df['User Id'] == user_id]['Amount'].sum()
    
    # Calculate the number of deposits and withdrawals
    num_deposits = deposit_df[deposit_df['User Id'] == user_id].shape[0]
    num_withdrawals = withdrawal_df[withdrawal_df['User Id'] == user_id].shape[0]
    
    # Calculate loyalty points using the provided formula
    loyalty_points[user_id] = (
        (0.01 * deposit_amount) +
        (0.005 * withdrawal_amount) +
        (0.001 * max(num_deposits - num_withdrawals, 0)) +
        (0.2 * games_played)
    )

# Create a DataFrame for loyalty points
loyalty_df = pd.DataFrame(loyalty_points.items(), columns=['User Id', 'Loyalty Points'])

# Identify top 50 players based on loyalty points
top_50 = loyalty_df.nlargest(50, 'Loyalty Points')

# Calculate total loyalty points and total games played of top 50 players
total_loyalty_points = top_50['Loyalty Points'].sum()
total_games_played = games_df[games_df['User ID'].isin(top_50['User Id'])]['Games Played'].sum()

# Allocate Rs 50000 bonus pool based on loyalty points and games played
total_bonus_pool = 50000
loyalty_weight = 0.7
games_weight = 0.3

top_50['Points Bonus'] = (top_50['Loyalty Points'] / total_loyalty_points) * total_bonus_pool * loyalty_weight
top_50['Games Bonus'] = (top_50['Loyalty Points'] / total_games_played) * total_bonus_pool * games_weight

# Calculate total bonus for each player
top_50['Total Bonus'] = top_50['Points Bonus'] + top_50['Games Bonus']

# Display the top players with their bonus allocations
print(top_50[['User Id', 'Loyalty Points', 'Total Bonus']])

     User Id  Loyalty Points   Total Bonus
634      634       83843.325  47772.048916
99        99       23665.737  13484.206949
672      672       22757.780  12966.873384
212      212       22199.282  12648.653731
740      740       19211.824  10946.467067
566      566       19153.755  10913.380651
714      714       16764.234   9551.885099
421      421       15446.460   8801.046985
369      369       14438.444   8226.702043
30        30       14053.375   8007.298350
587      587       13638.881   7771.128951
222      222       13348.803   7605.849003
352      352       13040.650   7430.270325
365      365       12855.075   7324.533846
920      920       12535.800   7142.618101
162      162       12483.600   7112.875710
415      415       12304.415   7010.780110
569      569       12285.423   6999.958894
786      786       12097.216   6892.722760
2          2       12040.475   6860.393009
238      238       11638.838   6631.549241
992      992       11627.777   6625.246931
28        2

## loyalty point formula is fair or unfair?

#### Fair Aspects of the Formula

1. Encourages Active Participation: The formula rewards players for their activity on the platform, including the number of games played, deposits made, and withdrawals. This promotes a vibrant gaming community where players are incentivized to engage regularly.
2. Balances Different Activities:By incorporating various actions (deposits, withdrawals, and games played), the formula recognizes multiple facets of player engagement. This can help to balance the interests of different types of players, whether they are more focused on gaming or financial transactions.
3. The points awarded per action are relatively small, meaning players must engage consistently over time to accumulate significant loyalty points. This encourages ongoing participation rather than a focus on single large transactions.

#### Unfair Aspects of the Formula

1. The current formula assigns significant weight to deposits (0.01 per unit), which can lead to situations where players with larger financial resources accumulate points more easily than those with less to invest. This might create a disparity where wealthier players can dominate the leaderboard without necessarily being more engaged.
2. While games played are considered, the weight (0.2) may not be sufficient to balance out the influence of large deposits. Players who engage actively but do not deposit much may feel undervalued, potentially discouraging them from playing.
3. Players could exploit the system by making large deposits without a corresponding level of engagement, allowing them to accumulate points without genuinely participating in the gaming experience.
4. Players who might not have the means to deposit large sums may feel alienated if their engagement does not translate into competitive loyalty points. This could lead to a decrease in overall player retention.


Overall, while the loyalty point formula has elements that promote fairness by rewarding diverse player activities, it has significant shortcomings regarding how it values those activities. The heavy weighting on deposits can create inequities, particularly for less affluent players or those who prioritize gaming over financial transactions.

### conclusion 

The formula could be considered partially **unfair** due to its current structure that favors financial investment over genuine engagement. Adjustments to balance the weight of deposits against gaming activity would create a more equitable system that promotes participation and retention across a broader range of players.

##  Way to make the loyalty point formula more robust?		

To make the loyalty point formula more robust, here are some adjustments that would create a more balanced and fair system for rewarding player activity and encourage genuine engagement on the platform:

**1. Rebalance Weighting of Actions**
- **Reduce Deposit Weight:** Lower the deposit weight from 0.01 to something like 0.005. This reduces the heavy influence of deposits in point accumulation, ensuring that financial contributions are valued but not overly dominant.
- **Increase Games Played Weight:** Increase the weight of games played from 0.2 to 0.3 or higher. This puts greater emphasis on genuine gaming engagement, rewarding players for participating in the platform’s core activity.

**2. Add a Consistency Bonus**
- **Introduce Points for Daily Activity Streaks:** Award bonus points for players who play games consistently over multiple consecutive days. For example, award an extra 1.5 loyalty points for each 3-day streak, encouraging ongoing engagement and helping retain users.
- **Reward Variety:** Include points for engaging in different types of games or multiplayer modes, incentivizing players to explore more of the platform.

**3. Limit Points for High Deposits and Withdrawals**
- **Introduce a Cap on Deposit Points per Month:** Place an upper limit on the loyalty points that can be earned through deposits in a single month. This prevents wealthier players from monopolizing the leaderboard simply through large deposits.
- **Apply Diminishing Returns:** Reduce the points awarded for deposits or withdrawals above a certain threshold to encourage fairer accumulation. For instance, a deposit above ₹10,000 might earn points at 0.005 rather than 0.01.

**4. Reward Retention and Long-Term Engagement**
- **Create a Tiered Loyalty System:** Award bonus points for long-term players. For instance, players who remain active on the platform for over 6 months could earn a 5% boost in points for games played. This encourages loyalty beyond high spending.
- **Recognize Low-Activity Players:** Offer small rewards or incentives to less active players, encouraging them to engage more frequently without making the leaderboard inaccessible to them.

**5. Account for Skill or Performance in Games**
- **Introduce a Performance-Based Component:** Award extra points for players who perform well in games, such as scoring high or winning consistently. This way, the formula rewards not only participation but also skill, creating a more merit-based system.

### Revised Loyalty Formula Example

Loyalty_Points = (0.005 * Deposit_Amount) + (0.003 * Withdrawal_Amount)

                + (0.001 * max(#deposit - #withdrawal, 0))
                
                + (0.3 * Number_of_Games_Played)
                
                + (1.5 * Daily_Streaks) + (Bonus for Performance/Skill)


This revised structure would better balance different types of engagement and create a more inclusive, skill-based, and sustainable points system. The goal is to make the leaderboard attainable for a wider range of players, encouraging both financial contributions and active participation without allowing one factor to dominate the loyalty points calculation.